In [ ]:
import os
import matplotlib.pyplot as plt
import cv2
import matplotlib.image as mpimg
from PIL import Image
from sklearn.model_selection import train_test_split

In [ ]:
data_list = os.listdir('dataset')
print(data_list)

In [ ]:
len(data_list)

In [ ]:
img = mpimg.imread('dataset/cat (1).jpg')
plt.imshow(img)
plt.show()

run this if resized image is not present in folder

In [ ]:
for i in data_list:
    img = Image.open('dataset/' + i)
    resized_img = img.resize((224,224))
    resized_img = resized_img.convert('RGB')
    resized_img.save('resized_dataset/' + i)

In [ ]:
dataset = os.listdir('resized_dataset')

In [ ]:
dataset

creating label

In [ ]:
Y = []
for i in dataset:
    if i[:3] == 'cat':
        Y.append(0)
    else:
        Y.append(1)

convert all resized iamge to numpy array

In [ ]:
import glob
import numpy as np

In [ ]:
dog_cat_images = np.asarray([cv2.imread('resized_dataset/'+file) for file in dataset])

In [ ]:
X = dog_cat_images
Y = np.asarray(Y)

shuffle

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
random_index = np.random.permutation(len(X))
X = X[random_index]
Y = Y[random_index]

split dataset

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, random_state=2)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
unique_elements, counts = np.unique(y_train, return_counts=True)

In [ ]:
for element, count in zip(unique_elements, counts):
    print(f"Element: {element}, Count: {count}")

scale image by divideing by 255

In [ ]:
x_test = x_test / 255

In [ ]:
x_train = x_train / 255

In [ ]:
x_train[0]

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
mobilenet_model = 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4'

pretrained_model = hub.KerasLayer(mobilenet_model, input_shape=(224,224,3), trainable=False)

In [ ]:
num_of_classes = 2

model = tf.keras.Sequential([
    
    pretrained_model,
    tf.keras.layers.Dense(num_of_classes)

])

model.summary()

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['acc']
)

In [ ]:
model.fit(x_train, y_train, epochs=5)

accuracy of model : 99.3%

In [ ]:
score, acc = model.evaluate(x_test, y_test)
print('Test Loss =', score)
print('Test Accuracy =', acc)

accuracy on test data is : 98%

In [ ]:
from tensorflow import keras

In [ ]:
keras.models.save_model(model, 'dogcat.keras')

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
y_pred_label = [np.argmax(i) for i in y_pred]
print(y_pred_label)

In [ ]:
con_mat = tf.math.confusion_matrix(y_test, y_pred_label)

In [ ]:
import seaborn as sns
plt.figure(figsize=(3,2))
sns.heatmap(con_mat, annot=True, fmt='d', cmap='Blues')
plt.xlabel('pridicted label')
plt.ylabel('actual label')
plt.show()

In [ ]:
# files = os.listdir('rest')
# for file in range(12500,13500):
#     input_image_path = 'rest/' + files[file]

#     input_image = cv2.imread(input_image_path)
#     try:
#         input_image_resize = cv2.resize(input_image, (224, 224))
#     except Exception as e:
#         continue
#     input_image_resize = input_image_resize / 255

#     image_reshaped = np.reshape(input_image_resize, [1,224,224,3])

#     input_prediction = model.predict(image_reshaped)

#     input_pred_label = np.argmax(input_prediction)

#     if input_pred_label == 0:
#         print("The Image is of Cat")
#     elif input_pred_label == 1:
#         print("The Image is of Dog.")

In [43]:
input_image_path = 'cat1.jpg'

input_image = cv2.imread(input_image_path)

input_image_resize = cv2.resize(input_image, (224, 224))

input_image_resize = input_image_resize / 255

image_reshaped = np.reshape(input_image_resize, [1,224,224,3])

input_prediction = model.predict(image_reshaped)

input_pred_label = np.argmax(input_prediction)

if input_pred_label == 0:
    print("The Image is of Cat")
elif input_pred_label == 1:
    print("The Image is of Dog.")

1/1 [==============================] - 0s 46ms/step
The Image is of Cat
